In [15]:
from wav2vec2 import Wav2Vec2Config, Wav2Vec2ForCTC, Wav2Vec2Config

import tensorflow as tf

In [16]:
config = Wav2Vec2Config()
model = Wav2Vec2ForCTC(config)

In [17]:
# batch = tf.ones((1, 1024), dtype=tf.float32)
# model(batch)

In [18]:
[v.name for v in model.variables]

['wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/layer_norm/gamma:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/layer_norm/beta:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/1/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/2/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/3/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/4/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/5/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/6/conv/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_projection/layer_norm/gamma:0',
 'wav2vec-ctc/wav2vec2/feature_projection/layer_norm/beta:0',
 'wav2vec-ctc/wav2vec2/feature_projection/projection/kernel:0',
 'wav2vec-ctc/wav2vec2/feature_projection/projection/bias:0',
 'wav2vec-ctc/wav2vec2/encoder/layer_norm/gamma:0',
 'wav2vec-ctc/wav2vec2/encoder/layer_norm/beta:0'

In [19]:
from transformers import Wav2Vec2ForCTC as HFWav2Vec2ForCTC

hf_model = HFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
model.variables[0].value

<bound method BaseResourceVariable.value of <tf.Variable 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/conv/kernel:0' shape=(10, 1, 512) dtype=float32, numpy=
array([[[ 0.03120919, -0.01594949,  0.02279448, ..., -0.02652317,
         -0.02737866, -0.00334822]],

       [[ 0.01261151, -0.02343632,  0.01583271, ...,  0.02090617,
          0.02896192, -0.00599178]],

       [[ 0.017125  ,  0.0146711 ,  0.00668287, ..., -0.01111716,
          0.02802647,  0.01305397]],

       ...,

       [[ 0.02383802,  0.00751015,  0.02661301, ..., -0.01388671,
         -0.02314447, -0.0341196 ]],

       [[ 0.00730148,  0.0042468 , -0.0157556 , ..., -0.02352146,
          0.02210403, -0.02011244]],

       [[-0.00910099,  0.01446106,  0.02493895, ...,  0.00298769,
          0.03198862, -0.02054293]]], dtype=float32)>>

In [27]:
model.load_weights("dummy")

In [44]:
model.get_weights()[0] = tf.ones((10,1,512))

In [47]:
model.get_weights()[0]

array([[[ 0.03120919, -0.01594949,  0.02279448, ..., -0.02652317,
         -0.02737866, -0.00334822]],

       [[ 0.01261151, -0.02343632,  0.01583271, ...,  0.02090617,
          0.02896192, -0.00599178]],

       [[ 0.017125  ,  0.0146711 ,  0.00668287, ..., -0.01111716,
          0.02802647,  0.01305397]],

       ...,

       [[ 0.02383802,  0.00751015,  0.02661301, ..., -0.01388671,
         -0.02314447, -0.0341196 ]],

       [[ 0.00730148,  0.0042468 , -0.0157556 , ..., -0.02352146,
          0.02210403, -0.02011244]],

       [[-0.00910099,  0.01446106,  0.02493895, ...,  0.00298769,
          0.03198862, -0.02054293]]], dtype=float32)

In [50]:
model.variables[:2]

[<tf.Variable 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/conv/kernel:0' shape=(10, 1, 512) dtype=float32, numpy=
 array([[[ 0.03120919, -0.01594949,  0.02279448, ..., -0.02652317,
          -0.02737866, -0.00334822]],
 
        [[ 0.01261151, -0.02343632,  0.01583271, ...,  0.02090617,
           0.02896192, -0.00599178]],
 
        [[ 0.017125  ,  0.0146711 ,  0.00668287, ..., -0.01111716,
           0.02802647,  0.01305397]],
 
        ...,
 
        [[ 0.02383802,  0.00751015,  0.02661301, ..., -0.01388671,
          -0.02314447, -0.0341196 ]],
 
        [[ 0.00730148,  0.0042468 , -0.0157556 , ..., -0.02352146,
           0.02210403, -0.02011244]],
 
        [[-0.00910099,  0.01446106,  0.02493895, ...,  0.00298769,
           0.03198862, -0.02054293]]], dtype=float32)>,
 <tf.Variable 'wav2vec-ctc/wav2vec2/feature_extractor/conv_layers/0/layer_norm/gamma:0' shape=(512,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
    

In [51]:
model.load_weights("dummy.h5")